<a href="https://colab.research.google.com/github/jeffreyfeng99/SYDE_522_A3/blob/master/SYDE_522_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Submission Notes
Group members:

Jeffrey Feng, 20704800

Joey Kuang, 20726074

# Import statements

In [ ]:
import os
import pandas as pd
import numpy as np
import random
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Function
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torchvision import datasets
from torchvision import models
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data loader

In [ ]:
dataset_root = '/content/drive/MyDrive/4B/SYDE-522/data'
source_dataset_name = 'train_set'
target_dataset_name = 'test_set'

source_image_root = os.path.join(dataset_root, source_dataset_name)
target_image_root = os.path.join(dataset_root, target_dataset_name)

In [ ]:
class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list=None, transform=None):
        self.root = data_root
        self.transform = transform

        # we only pass data_list if it's training set
        if data_list is not None:
            df = pd.read_csv(data_list)
            self.img_paths = df['dir'].to_list()

            if 'label2' in df.columns:
                self.img_labels = df['label2'].to_list()
            else: 
                self.img_labels = ['0' for i in range(len(self.img_paths))]
        else:
            # Walk through test folder - we don't need labels
            self.img_paths = [f for root,dirs,files in os.walk(data_root) for f in files if f.endswith('.png')]
            self.img_labels = ['0' for i in range(len(self.img_paths))]

        self.n_data = len(self.img_paths)

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item%self.n_data], self.img_labels[item%self.n_data]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels

    def __len__(self):
        return self.n_data

# Model definition

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel, self).__init__()
        # self.feature = nn.Sequential()
        # self.feature.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
        # self.feature.add_module('f_bn1', nn.BatchNorm2d(64))
        # self.feature.add_module('f_pool1', nn.MaxPool2d(2))
        # self.feature.add_module('f_relu1', nn.ReLU(True))
        # self.feature.add_module('f_conv2', nn.Conv2d(64, 128, kernel_size=3))
        # self.feature.add_module('f_bn2', nn.BatchNorm2d(128))
        # self.feature.add_module('f_drop1', nn.Dropout2d())
        # self.feature.add_module('f_pool2', nn.MaxPool2d(2))
        # self.feature.add_module('f_relu2', nn.ReLU(True))
        # self.feature.add_module('f_conv3', nn.Conv2d(128, 256, kernel_size=3))
        # self.feature.add_module('f_bn3', nn.BatchNorm2d(256))
        # self.feature.add_module('f_drop3', nn.Dropout2d())
        # self.feature.add_module('f_pool3', nn.MaxPool2d(2))
        # self.feature.add_module('f_relu4', nn.ReLU(True))
        # self.feature.add_module('f_conv4', nn.Conv2d(256, 256, kernel_size=3))
        # self.feature.add_module('f_bn4', nn.BatchNorm2d(256))
        # self.feature.add_module('f_drop4', nn.Dropout2d())
        # self.feature.add_module('f_pool4', nn.MaxPool2d(2))
        # self.feature.add_module('f_relu4', nn.ReLU(True))
        # self.feature.add_module('f_conv5', nn.Conv2d(256, 512, kernel_size=3))
        # self.feature.add_module('f_bn5', nn.BatchNorm2d(512))
        # self.feature.add_module('f_drop5', nn.Dropout2d())
        # self.feature.add_module('f_pool5', nn.MaxPool2d(2))
        # self.feature.add_module('f_relu5', nn.ReLU(True))

        self.feature = models.resnet18(pretrained=True) 
        self.feature.fc = nn.Identity()

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(512, 100))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout2d())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 7))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax(dim=1))

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(512, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 4))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 224, 224)
        feature = self.feature(input_data)
        feature = feature.view(-1, 512)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

# Test pipeline

In [ ]:
def test(net, dataset_name, epoch):

    image_root = os.path.join(dataset_root, dataset_name)

    cuda = True
    cudnn.benchmark = True
    batch_size = 64
    image_size = 224 # original 227, but pretrained resnet18 takes in 224
    alpha = 0

    # load data

    img_transform_source = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.1307,), std=(0.3081,))
    ])

    img_transform_target = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])

    train_label_list = os.path.join(dataset_root, 'train_labels.csv')
    test_label_list = os.path.join(dataset_root, 'test_labels.csv')

    dataset_source = GetLoader(
        data_root=os.path.join(image_root, 'train_set'),
        data_list=train_label_list,
        transform=img_transform_target
    )

    dataloader_source = data.DataLoader(
        dataset=dataset_source,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8,
        drop_last=True)

    dataset_target = GetLoader(
        data_root=os.path.join(image_root, 'test_set'),
        transform=img_transform_target
    )

    dataloader_target = data.DataLoader(
        dataset=dataset_target,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8,
        drop_last=True
    )

    """ testing """

    # my_net = torch.load(os.path.join('output', 'model_epoch_' + str(epoch) + '.pth'))
    my_net = net.eval() #my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader_source)
    data_target_iter = iter(dataloader_source)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        class_output, _ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))

In [ ]:
cuda = True
cudnn.benchmark = True
lr = 1e-3
batch_size = 64
image_size = 224 #227
n_epoch = 100

manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# load data

img_transform_source = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

img_transform_target = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

train_label_list = os.path.join(dataset_root, 'train_labels.csv')
test_label_list = os.path.join(dataset_root, 'test_labels.csv')

dataset_source = GetLoader(
    data_root=os.path.join(source_image_root, 'train_set'),
    data_list=train_label_list,
    transform=img_transform_target
)

dataloader_source = data.DataLoader(
    dataset=dataset_source,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    drop_last=True)

dataset_target = GetLoader(
    data_root=os.path.join(target_image_root, 'test_set'),
    transform=img_transform_target
)

dataloader_target = data.DataLoader(
    dataset=dataset_target,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    drop_last=True)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# load model

my_net = CNNModel()

# setup optimizer

optimizer = optim.Adam(my_net.parameters(), lr=lr)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True

In [ ]:
# training

for epoch in tqdm(range(n_epoch)):

    len_dataloader = max(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)

        class_output, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        if i == len(dataloader_target):
            data_target_iter = iter(dataloader_target)
        data_target = data_target_iter.next()
        t_img, _ = data_target

        batch_size = len(t_img)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1

        print('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
            % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
                err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

    # torch.save(my_net, 'output/model_epoch_{}.pth'.format(epoch))
    test(my_net, source_dataset_name, epoch)
    my_net.train()

print('done')

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: 0, [iter: 1 / all 94], err_s_label: 1.923123, err_s_domain: 1.755530, err_t_domain: 1.339389
epoch: 0, [iter: 2 / all 94], err_s_label: 1.657065, err_s_domain: 1.734935, err_t_domain: 1.281352
epoch: 0, [iter: 3 / all 94], err_s_label: 1.695674, err_s_domain: 1.675541, err_t_domain: 1.245419
epoch: 0, [iter: 4 / all 94], err_s_label: 1.489899, err_s_domain: 1.612949, err_t_domain: 1.185603
epoch: 0, [iter: 5 / all 94], err_s_label: 1.370999, err_s_domain: 1.569276, err_t_domain: 1.160763
epoch: 0, [iter: 6 / all 94], err_s_label: 1.416976, err_s_domain: 1.525028, err_t_domain: 1.116246
epoch: 0, [iter: 7 / all 94], err_s_label: 1.341486, err_s_domain: 1.462768, err_t_domain: 1.066229
epoch: 0, [iter: 8 / all 94], err_s_label: 1.252724, err_s_domain: 1.412463, err_t_domain: 1.054663
epoch: 0, [iter: 9 / all 94], err_s_label: 1.282753, err_s_domain: 1.357559, err_t_domain: 1.028062
epoch: 0, [iter: 10 / all 94], err_s_label: 1.103857, err_s_domain: 1.342712, err_t_domain: 0.980194

  1%|          | 1/100 [02:00<3:19:36, 120.98s/it]

epoch: 0, accuracy of the train_set dataset: 0.705785
epoch: 1, [iter: 1 / all 94], err_s_label: 0.371794, err_s_domain: 0.669147, err_t_domain: 0.622111
epoch: 1, [iter: 2 / all 94], err_s_label: 0.585315, err_s_domain: 0.687866, err_t_domain: 0.659537
epoch: 1, [iter: 3 / all 94], err_s_label: 0.478940, err_s_domain: 0.714373, err_t_domain: 0.693243
epoch: 1, [iter: 4 / all 94], err_s_label: 0.427667, err_s_domain: 0.666195, err_t_domain: 0.652986
epoch: 1, [iter: 5 / all 94], err_s_label: 0.502760, err_s_domain: 0.650688, err_t_domain: 0.673831
epoch: 1, [iter: 6 / all 94], err_s_label: 0.577101, err_s_domain: 0.647274, err_t_domain: 0.710873
epoch: 1, [iter: 7 / all 94], err_s_label: 0.816115, err_s_domain: 0.679960, err_t_domain: 0.668504
epoch: 1, [iter: 8 / all 94], err_s_label: 0.674535, err_s_domain: 0.639911, err_t_domain: 0.697671
epoch: 1, [iter: 9 / all 94], err_s_label: 0.493641, err_s_domain: 0.662618, err_t_domain: 0.660255
epoch: 1, [iter: 10 / all 94], err_s_label: 0.

  2%|▏         | 2/100 [03:53<3:09:04, 115.76s/it]

...

epoch: 57, accuracy of the train_set dataset: 0.162733
epoch: 58, [iter: 1 / all 94], err_s_label: 0.009439, err_s_domain: 0.669782, err_t_domain: 0.727780
epoch: 58, [iter: 2 / all 94], err_s_label: 0.026978, err_s_domain: 0.684037, err_t_domain: 0.720707
epoch: 58, [iter: 3 / all 94], err_s_label: 0.002961, err_s_domain: 0.685134, err_t_domain: 0.713198
epoch: 58, [iter: 4 / all 94], err_s_label: 0.005490, err_s_domain: 0.682363, err_t_domain: 0.709282
epoch: 58, [iter: 5 / all 94], err_s_label: 0.002504, err_s_domain: 0.695365, err_t_domain: 0.703819
epoch: 58, [iter: 6 / all 94], err_s_label: 0.002804, err_s_domain: 0.693141, err_t_domain: 0.700400
epoch: 58, [iter: 7 / all 94], err_s_label: 0.040493, err_s_domain: 0.705467, err_t_domain: 0.693756
epoch: 58, [iter: 8 / all 94], err_s_label: 0.005066, err_s_domain: 0.709806, err_t_domain: 0.693431
epoch: 58, [iter: 9 / all 94], err_s_label: 0.082797, err_s_domain: 0.718717, err_t_domain: 0.689643
epoch: 58, [iter: 10 / all 94], err_

 59%|█████▉    | 59/100 [1:48:59<1:14:52, 109.58s/it]

epoch: 58, accuracy of the train_set dataset: 0.211602
epoch: 59, [iter: 1 / all 94], err_s_label: 0.243967, err_s_domain: 0.693499, err_t_domain: 0.675149
epoch: 59, [iter: 2 / all 94], err_s_label: 0.015017, err_s_domain: 0.688158, err_t_domain: 0.680584
epoch: 59, [iter: 3 / all 94], err_s_label: 0.010068, err_s_domain: 0.676816, err_t_domain: 0.688104
epoch: 59, [iter: 4 / all 94], err_s_label: 0.004166, err_s_domain: 0.681295, err_t_domain: 0.687418
epoch: 59, [iter: 5 / all 94], err_s_label: 0.000560, err_s_domain: 0.675428, err_t_domain: 0.694470
epoch: 59, [iter: 6 / all 94], err_s_label: 0.082661, err_s_domain: 0.675794, err_t_domain: 0.705210
epoch: 59, [iter: 7 / all 94], err_s_label: 0.008896, err_s_domain: 0.678655, err_t_domain: 0.712939
epoch: 59, [iter: 8 / all 94], err_s_label: 0.162359, err_s_domain: 0.672274, err_t_domain: 0.714733
epoch: 59, [iter: 9 / all 94], err_s_label: 0.004140, err_s_domain: 0.658161, err_t_domain: 0.719230
epoch: 59, [iter: 10 / all 94], err_

 59%|█████▉    | 59/100 [1:49:47<1:16:17, 111.65s/it]


KeyboardInterrupt: ignored